In [3]:
from dotenv import load_dotenv
from pyspark.sql import SparkSession, functions as F

load_dotenv()

spark = SparkSession.builder.appName("TlcTripsAnalysis").getOrCreate()
df = spark.read.parquet("s3a://datalake-prd-tlc-trips/trusted-zone/yellow_tripdata/", header=True, inferSchema=False)

In [4]:
df = df.select(
    F.col('VendorID').alias('cd_empresa'),
    F.col('VendorDesc').alias('ds_empresa'),
    F.col('passenger_count').alias('qt_passageiros'),
    F.col('total_amount').alias('vl_total'),
    F.col('tpep_pickup_datetime').alias('ts_inicio_corrida'),
    F.col('tpep_dropoff_datetime').alias('ts_fim_corrida'),
    F.col('trip_distance').alias('vl_distancia_mi'),
    F.col('RatecodeID').alias('cd_tarifa'),
    F.col('RatecodeDesc').alias('ds_tarifa'),
    F.col("store_and_fwd_flag").alias("fl_transmissao"),
    F.col("store_and_fwd_desc").alias("ds_transmissao"),
    F.col('PULocationID').alias('cd_zona_embarque'),
    F.col('DOLocationID').alias('cd_zona_desembarque'),
    F.col('payment_type').alias('cd_pagamento'),
    F.col('payment_desc').alias('ds_pagamento'),
    F.col('fare_amount').alias('vl_tarifa_base'),
    F.col('extra').alias('vl_extra'),
    F.col('mta_tax').alias('vl_mta_tax'),
    F.col('tip_amount').alias('vl_gorjeta'),
    F.col('tolls_amount').alias('vl_pedagio'),
    F.col('improvement_surcharge').alias('vl_sobretaxa_melhoria'),
    F.col('congestion_surcharge').alias('vl_sobretaxa_congestionamento'),
    F.col('airport_fee').alias('vl_taxa_aeroporto'),
)

print(df.count())
print(df.dropDuplicates().count())
# print(len(df.columns))
print(df.describe().show())

16186386
16186386
23
+-------+-------------------+--------------------+------------------+------------------+------------------+-----------------+-------------+--------------+--------------------+------------------+-------------------+------------------+------------+-----------------+------------------+-------------------+------------------+-----------------+---------------------+-----------------------------+-----------------+
|summary|         cd_empresa|          ds_empresa|    qt_passageiros|          vl_total|   vl_distancia_mi|        cd_tarifa|    ds_tarifa|fl_transmissao|      ds_transmissao|  cd_zona_embarque|cd_zona_desembarque|      cd_pagamento|ds_pagamento|   vl_tarifa_base|          vl_extra|         vl_mta_tax|        vl_gorjeta|       vl_pedagio|vl_sobretaxa_melhoria|vl_sobretaxa_congestionamento|vl_taxa_aeroporto|
+-------+-------------------+--------------------+------------------+------------------+------------------+-----------------+-------------+--------------+---

In [5]:
df.printSchema()

vol_total = df.count()
dt_min, dt_max = df.select(F.min("ts_inicio_corrida"), F.max("ts_fim_corrida")).first()
nulos_chave = (
    df.select([
        F.sum(F.col(c).isNull().cast("int")).alias(c) 
        for c in ["ts_inicio_corrida","ts_fim_corrida","vl_total","qt_passageiros"]
    ])
    .toPandas()
)

vol_total, dt_min, dt_max, nulos_chave


root
 |-- cd_empresa: integer (nullable = true)
 |-- ds_empresa: string (nullable = true)
 |-- qt_passageiros: integer (nullable = true)
 |-- vl_total: float (nullable = true)
 |-- ts_inicio_corrida: timestamp (nullable = true)
 |-- ts_fim_corrida: timestamp (nullable = true)
 |-- vl_distancia_mi: float (nullable = true)
 |-- cd_tarifa: integer (nullable = true)
 |-- ds_tarifa: string (nullable = true)
 |-- fl_transmissao: string (nullable = true)
 |-- ds_transmissao: string (nullable = true)
 |-- cd_zona_embarque: integer (nullable = true)
 |-- cd_zona_desembarque: integer (nullable = true)
 |-- cd_pagamento: integer (nullable = true)
 |-- ds_pagamento: string (nullable = true)
 |-- vl_tarifa_base: float (nullable = true)
 |-- vl_extra: float (nullable = true)
 |-- vl_mta_tax: float (nullable = true)
 |-- vl_gorjeta: float (nullable = true)
 |-- vl_pedagio: float (nullable = true)
 |-- vl_sobretaxa_melhoria: float (nullable = true)
 |-- vl_sobretaxa_congestionamento: float (nullable =

(16186386,
 datetime.datetime(2000, 12, 31, 21, 6, 49),
 datetime.datetime(2023, 9, 5, 15, 24, 21),
    ts_inicio_corrida  ts_fim_corrida  vl_total  qt_passageiros
 0                  0               0         0          428665)

In [6]:
df_clean = df.where(
    (F.col("vl_total") >= 0) &
    (F.col("ts_inicio_corrida") >= '2023-01-01') &
    (F.col("ts_inicio_corrida") <= '2023-05-31') &
    (F.col("qt_passageiros") >= 0) &
    (F.col("vl_distancia_mi") >= 0)
)

before, after = df.count(), df_clean.count()
removidos = before - after
{"antes": before, "depois": after, "removidos": removidos, "perc_removido_%": round(removidos*100.0/max(before,1), 4)}


{'antes': 16186386,
 'depois': 15493528,
 'removidos': 692858,
 'perc_removido_%': 4.2805}

In [7]:
df_enr = (
    df_clean
    .withColumn("ano", F.year("ts_inicio_corrida"))
    .withColumn("mes", F.month("ts_inicio_corrida"))
    .withColumn("hora", F.hour("ts_inicio_corrida"))
)

df_enr.select("ano","mes","hora").summary("count","min","max").toPandas()


,summary,ano,mes,hora
0,count,15493528,15493528,15493528
1,min,2023,1,0
2,max,2023,5,23


## Resposta das perguntas

#### 1: Qual a média de valor total (total\_amount) recebido em um mês considerando todos os yellow táxis da frota?

In [14]:
df_base = df_enr

media_mensal_total = (
    df_base.alias('df_base').groupBy("ano", "mes")
           .agg(F.avg("vl_total").alias("media_vl_total"))
           .orderBy("ano", "mes")
)

media_geral = media_mensal_total.agg(
    F.avg("media_vl_total").alias("media_geral_vl_total")).toPandas()

display(media_mensal_total.toPandas())
display(media_geral)

# Mesma query se fosse SQL:
# SELECT
#     ano,
#     mes,
#     AVG(vl_total) AS media_vl_total
# FROM
#     df_base
# GROUP BY
#     ano, mes
# ORDER BY
#     ano, mes;

,ano,mes,media_vl_total
0,2023,1,27.406889
1,2023,2,27.318139
2,2023,3,28.213547
3,2023,4,28.756875
4,2023,5,29.375385


,media_geral_vl_total
0,28.214167


#### 2: Qual a média de passageiros (passenger\_count) por cada hora do dia que pegaram táxi no mês de maio considerando todos os táxis da frota?

In [ ]:
maio = df_enr.alias('df_enr').where((F.col("mes")==5) & (F.col("ano")==2023))

media_passageiros_por_hora_maio = (
    maio.groupBy("hora")
        .agg(F.avg("qt_passageiros").alias("media_qt_passageiros"))
        .orderBy("hora")
)

media_passageiros_por_hora_maio.toPandas()

# Mesma query se fosse em SQL:
# SELECT
#     hora,
#     AVG(qt_passageiros) AS media_qt_passageiros
# FROM
#     df_enr
# WHERE
#     mes = 5
#     AND ano = 2023
# GROUP BY
#     hora
# ORDER BY
#     hora;

,hora,media_qt_passageiros
0,0,1.437775
1,1,1.390284
2,2,1.265884
3,3,1.235409
4,4,1.253372
5,5,1.266281
6,6,1.284876
7,7,1.319951
8,8,1.335050
9,9,1.349327
